<a href="https://colab.research.google.com/github/elangbijak4/Riset-Smart-City/blob/main/Demo_EV_HEV_PHEV_dalam_Konteks_MLP_FL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# 🌐 Install dependencies (Google Colab)
!pip install flwr[simulation] torch torchvision -q

# 📦 Import library
import flwr as fl
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

# ✅ Define MLP architectures

class EVModel(nn.Module):
    def __init__(self):
        super(EVModel, self).__init__()
        self.fc1 = nn.Linear(10, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 1)  # Regression output

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

class HEVModel(nn.Module):
    def __init__(self):
        super(HEVModel, self).__init__()
        self.fc1 = nn.Linear(12, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 2)  # Classification: e.g. mode ICE vs Motor

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return F.log_softmax(self.fc3(x), dim=1)

class PHEVModel(nn.Module):
    def __init__(self):
        super(PHEVModel, self).__init__()
        self.fc1 = nn.Linear(15, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)  # Regression output: optimal EV mode ratio

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

# 📊 Generate dummy data for each vehicle
def generate_data(n_samples, input_size, output_type):
    X = torch.rand(n_samples, input_size)
    if output_type == "regression":
        y = torch.sum(X, dim=1, keepdim=True) + 0.1 * torch.randn(n_samples, 1)
    else:
        y = torch.randint(0, 2, (n_samples,))
    return TensorDataset(X, y)

# 🧠 Define Flower client
class VehicleClient(fl.client.NumPyClient):
    def __init__(self, model, train_loader, test_loader, is_classification=False):
        self.model = model
        self.train_loader = train_loader
        self.test_loader = test_loader
        self.is_classification = is_classification
        self.loss_fn = nn.NLLLoss() if is_classification else nn.MSELoss()
        self.optimizer = optim.Adam(model.parameters(), lr=0.01)

    def get_parameters(self, config):
        return [val.cpu().numpy() for val in self.model.state_dict().values()]

    def set_parameters(self, parameters):
        params_dict = zip(self.model.state_dict().keys(), parameters)
        state_dict = {k: torch.tensor(v) for k, v in params_dict}
        self.model.load_state_dict(state_dict, strict=True)

    def fit(self, parameters, config):
        self.set_parameters(parameters)
        self.model.train()
        for _ in range(1):
            for X, y in self.train_loader:
                self.optimizer.zero_grad()
                output = self.model(X)
                loss = self.loss_fn(output, y if self.is_classification else y)
                loss.backward()
                self.optimizer.step()
        return self.get_parameters(config={}), len(self.train_loader.dataset), {}

    def evaluate(self, parameters, config):
        self.set_parameters(parameters)
        self.model.eval()
        loss = 0
        correct = 0
        with torch.no_grad():
            for X, y in self.test_loader:
                output = self.model(X)
                loss += self.loss_fn(output, y if self.is_classification else y).item()
                if self.is_classification:
                    pred = output.argmax(dim=1, keepdim=True)
                    correct += pred.eq(y.view_as(pred)).sum().item()
        if self.is_classification:
            accuracy = correct / len(self.test_loader.dataset)
            return loss, len(self.test_loader.dataset), {"accuracy": accuracy}
        else:
            return loss, len(self.test_loader.dataset), {}

# ⚙️ Create each client and simulate FL

def client_fn(cid: str) -> fl.client.NumPyClient:
    if cid == "0":
        model = EVModel()
        dataset = generate_data(100, 10, "regression")
        train_loader = DataLoader(dataset, batch_size=16)
        return VehicleClient(model, train_loader, train_loader, is_classification=False)
    elif cid == "1":
        model = HEVModel()
        dataset = generate_data(100, 12, "classification")
        train_loader = DataLoader(dataset, batch_size=16)
        return VehicleClient(model, train_loader, train_loader, is_classification=True)
    elif cid == "2":
        model = PHEVModel()
        dataset = generate_data(100, 15, "regression")
        train_loader = DataLoader(dataset, batch_size=16)
        return VehicleClient(model, train_loader, train_loader, is_classification=False)

# 🚀 Start Flower simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=3,
    config=fl.server.ServerConfig(num_rounds=3),
)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 82.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.

	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=3, no round_timeout
2025-05-27 06:39:07,450	INFO worker.py:1771 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources

History (loss, distributed):
	round 1: 50.26929807662964
	round 2: 8.09563422203064
	round 3: 2.3778479993343353